In [23]:
import pandas as pd
import numpy as np
import time
import sys

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression  
from sklearn import svm

module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
if module_path not in sys.path:
    sys.path.append(module_path)
    
from integrating_embedding import *

import integrating_embedding
from imp import reload
reload(integrating_embedding)

module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
if module_path not in sys.path:
    sys.path.append(module_path)

import get_hospital_eicu


emb_path_trn_mimic = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/Train/result/emb_train_new_version(0303).npy'
emb_path_vld_mimic = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/Train/result/emb_valid_new_version(0303).npy'
# emb_path_event_mimic = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/Train/result/emb_event_new_version(0303).npy'

emb_path_trn_eicu = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/Train/result/emb_eicu_new_version(0303).npy'

mimic_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/mimic_analysis(new_version0229).csv.gz'
eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eicu_analysis(new_version0229).csv.gz'

mimic_train_emb, mimic_valid_emb, event = integrating_embedding.integrating(mimic_path, emb_path_trn_mimic, emb_path_vld_mimic, _, 'mimic')
mimic_train_emb = mimic_train_emb[mimic_train_emb['INDEX']=='CASE3_CASE4_DF'].reset_index(drop=True)
mimic_valid_emb = mimic_valid_emb[mimic_valid_emb['INDEX']=='CASE3_CASE4_DF'].reset_index(drop=True)
eicu_test_emb, event_eicu, hosp_id = integrating_embedding.integrating(eicu_path, emb_path_trn_eicu, _, _, 'eicu')
eicu_type = get_hospital_eicu.eicu_subgroup(eicu_test_emb)
unitadmitsource, unittype, unitstaytype = get_hospital_eicu.make_eicu_dataset(eicu_type[eicu_type['INDEX']=='CASE3_CASE4_DF'].reset_index(drop=True))

========== 데이터셋 분할 정보 ==========
데이터셋 비율: 학습 = 0.90, 테스트 = 0.10
학습셋 클래스 비율: Case
1.0    473941
2.0      5128
3.0     15571
4.0     15705
Name: count, dtype: int64
테스트셋 클래스 비율: Case
1.0    53103
2.0      512
3.0     1253
4.0     1271
Name: count, dtype: int64
--------------------------------------
========== 클래스 비율 ==========
학습셋 클래스 비율: 0.93:0.01:0.03:0.03
테스트셋 클래스 비율: 0.95:0.01:0.02:0.02
--------------------------------------
========== 환자 및 체류 정보 ==========
학습셋 환자 수: 2907
테스트셋 환자 수: 329
학습셋 체류 수: 2961
테스트셋 체류 수: 330
--------------------------------------
========== 실험 설정 ==========
분할 시드: 9040
학습 비율: 0.9
임계값: 0.05
--------------------------------------
========== 실행 결과 ==========
총 소요 시간(초): 0.38
시도한 시행 횟수: 0


In [109]:
module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/sub task prediction/'

if module_path not in sys.path:
    sys.path.append(module_path)

import incorperate_subtask_label
reload(incorperate_subtask_label)

<module 'incorperate_subtask_label' from 'c:\\Users\\DAHS\\Desktop\\ECP_CONT\\ECP_SCL\\Experiment(Supervised Learning)\\sub task prediction\\incorperate_subtask_label.py'>

In [110]:
mort_mimic_train, mort_mimic_valid, mort_eicu_test = incorperate_subtask_label.mortality_prediction_DATA(mimic_train_emb, mimic_valid_emb, eicu_test_emb)
los_mimic_train, los_mimic_valid, los_eicu_test = incorperate_subtask_label.LOS_prediction_DATA(mimic_train_emb, mimic_valid_emb, eicu_test_emb)
ards8h_mimic_train, ards8h_mimic_valid, ards8h_eicu_test = incorperate_subtask_label.ARDS8_prediction_DATA(mimic_train_emb, mimic_valid_emb, eicu_test_emb)
sic8h_mimic_train, sic8h_mimic_valid, sic8h_eicu_test = incorperate_subtask_label.SIC8_prediction_DATA(mimic_train_emb, mimic_valid_emb, eicu_test_emb)

In [111]:
drop_stayid = []
retain_stay_id = []
for stay_id, group in ards8h_mimic_train.groupby('stay_id'):
    if any(group['Annotation_ARDS'] == 'ARDS'):
        if all(group['Annotation_ARDS'] == 'ARDS'):
            drop_stayid.append(stay_id)
        else:
            retain_stay_id.append(stay_id)
            

for stay_id, group in ards8h_mimic_valid.groupby('stay_id'):
    if any(group['Annotation_ARDS'] == 'ARDS'):
        if all(group['Annotation_ARDS'] == 'ARDS'):
            drop_stayid.append(stay_id)
        else:
            retain_stay_id.append(stay_id)
            
for stay_id, group in ards8h_eicu_test.groupby('patientunitstayid'):
    if any(group['Annotation_ARDS'] == 'ARDS'):
        if all(group['Annotation_ARDS'] == 'ARDS'):
            drop_stayid.append(stay_id)
        else:
            retain_stay_id.append(stay_id)
            
ards8h_mimic_train = ards8h_mimic_train[(ards8h_mimic_train['stay_id'].isin(retain_stay_id))].reset_index(drop=True)
ards8h_mimic_valid = ards8h_mimic_valid[(ards8h_mimic_valid['stay_id'].isin(retain_stay_id))].reset_index(drop=True)
ards8h_eicu_test = ards8h_eicu_test[(ards8h_eicu_test['patientunitstayid'].isin(retain_stay_id))].reset_index(drop=True)

ARDS8h_event_train = ards8h_mimic_train[ards8h_mimic_train['Annotation_ARDS']=='ARDS']
ARDS8h_event_train['ARDS_next_8h']='event'
ARDS8h_event_valid = ards8h_mimic_valid[ards8h_mimic_valid['Annotation_ARDS']=='ARDS']
ARDS8h_event_valid['ARDS_next_8h']='event'
ARDS8h_event_test = ards8h_eicu_test[ards8h_eicu_test['Annotation_ARDS']=='ARDS']
ARDS8h_event_test['ARDS_next_8h']='event'


retain_stay_id = []
drop_stayid = []
for stay_id, group in sic8h_mimic_train.groupby('stay_id'):
    if any(group['Annotation_SIC'] == 'SIC'):
        if all(group['Annotation_SIC'] == 'SIC'):
            drop_stayid.append(stay_id)
        else:
            retain_stay_id.append(stay_id)
            
for stay_id, group in sic8h_mimic_valid.groupby('stay_id'):
    if any(group['Annotation_SIC'] == 'SIC'):
        if all(group['Annotation_SIC'] == 'SIC'):
            drop_stayid.append(stay_id)
        else:
            retain_stay_id.append(stay_id)
            
sic8h_mimic_train = sic8h_mimic_train[(sic8h_mimic_train['stay_id'].isin(retain_stay_id))].reset_index(drop=True)
sic8h_mimic_valid = sic8h_mimic_valid[(sic8h_mimic_valid['stay_id'].isin(retain_stay_id))].reset_index(drop=True)


SIC8h_event_train = sic8h_mimic_train[sic8h_mimic_train['Annotation_SIC']=='SIC']
SIC8h_event_train['SIC_next_8h']='event'
SIC8h_event_valid = sic8h_mimic_valid[sic8h_mimic_valid['Annotation_SIC']=='SIC']
SIC8h_event_valid['SIC_next_8h']='event'
SIC8h_event_test = sic8h_eicu_test[sic8h_eicu_test['Annotation_SIC']=='SIC']
SIC8h_event_test['SIC_next_8h']='event'

In [112]:
import split
from imp import reload
reload(split)

<module 'split' from 'c:\\Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation\\split.py'>

In [113]:
X_train_mort, y_train_mort, _ = split.split_X_Y_MORT(mort_mimic_train[~(mort_mimic_train['death']=='event')], mode = 'mimic')
X_valid_mort, y_valid_mort, valid_output_mort = split.split_X_Y_MORT(mort_mimic_valid[~(mort_mimic_valid['death']=='event')], mode = 'mimic')

X_train_los, y_train_los, _ = split.split_X_Y_LOS(los_mimic_train, mode = 'mimic')
X_valid_los, y_valid_los, valid_output_los = split.split_X_Y_LOS(los_mimic_valid, mode = 'mimic')

X_train_ards8h, y_train_ards8h, _ = split.split_X_Y_ARDS8h(ards8h_mimic_train[ards8h_mimic_train['Annotation_ARDS']=='no_ARDS'], mode = 'mimic')
X_valid_ards8h, y_valid_ards8h, valid_output_ards8h = split.split_X_Y_ARDS8h(ards8h_mimic_valid[ards8h_mimic_valid['Annotation_ARDS']=='no_ARDS'], mode = 'mimic')

X_train_sic8h, y_train_sic8h, _ = split.split_X_Y_SIC8h(sic8h_mimic_train[sic8h_mimic_train['Annotation_SIC']=='no_SIC'], mode = 'mimic')
X_valid_sic8h, y_valid_sic8h, valid_output_sic8h = split.split_X_Y_SIC8h(sic8h_mimic_valid[sic8h_mimic_valid['Annotation_SIC']=='no_SIC'], mode = 'mimic')

In [37]:
import Multiclass_evaluation
reload(Multiclass_evaluation)

<module 'Multiclass_evaluation' from 'c:\\Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation\\Multiclass_evaluation.py'>

In [38]:
mort_models, result_mort = Multiclass_evaluation.create_subtask(X_train_mort, y_train_mort, X_valid_mort, valid_output_mort, _,mode = 'mimic', type = 'binary', event_task = False)

|MIMIC-Validation|====================================================


,Model,Accuracy,AUPRC,F1 Score,Recall,Precision
0,lgbm,0.628429,0.614556,0.406375,0.294798,0.653846
1,rf,0.660848,0.593045,0.488722,0.375723,0.698925
2,dt,0.603491,0.478109,0.430108,0.346821,0.566038
3,lr,0.680798,0.644475,0.573333,0.497110,0.677165


----------------------------------------------------------------------


In [94]:
result_mort.to_excel(excel_writer='mort_proposed_method.xlsx')

In [71]:
los_models, result_los = Multiclass_evaluation.create_subtask(X_train_los, y_train_los, X_valid_los, valid_output_los, _, mode = 'mimic', type = 'multi', event_task = False)

|MIMIC-Validation|====================================================


,Model,Accuracy,Macro AUPRC,F1 Score,Macro averaged precision,Macro averaged Recall
0,lgbm,0.5729,0.666667,0.5239,0.6018,0.5109
1,rf,0.5305,0.632524,0.4002,0.4327,0.4043
2,dt,0.4671,0.583828,0.3663,0.3667,0.3757
3,lr,0.5765,0.666667,0.5518,0.5366,0.5837


----------------------------------------------------------------------


In [96]:
result_los.to_excel(excel_writer='los_proposed_method.xlsx')

In [129]:
reload(Multiclass_evaluation) 

<module 'Multiclass_evaluation' from 'c:\\Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation\\Multiclass_evaluation.py'>

In [139]:
ards8h_models, result_ards8h = Multiclass_evaluation.create_subtask(X_train_ards8h, y_train_ards8h, X_valid_ards8h, valid_output_ards8h, ARDS8h_event_valid, mode = 'mimic', type = 'binary', event_task='ARDS 8h')

|MIMIC-Validation|====================================================


,Model,AUPRC,ARDS score,Recall,Precision,Grecall
0,lgbm,0.428676,0.629032,0.8333,0.434211,0.785714
1,rf,0.485749,0.623839,1.0000,0.462687,0.738095
2,dt,0.691615,0.675074,1.0000,0.472973,0.833333
3,lr,0.657576,0.736544,1.0000,0.487805,0.952381


----------------------------------------------------------------------


In [141]:
result_ards8h.to_excel(excel_writer='ards8h_proposed_method.xlsx')

In [142]:
X_train_sic8h, y_train_sic8h, _ = split.split_X_Y_SIC8h(sic8h_mimic_train, mode = 'mimic')
X_valid_sic8h, y_valid_sic8h, valid_output_sic8h = split.split_X_Y_SIC8h(sic8h_mimic_valid[sic8h_mimic_valid['Annotation_SIC']=='no_SIC'], mode = 'mimic')

In [143]:
sic8h_models, result_sic8h = Multiclass_evaluation.create_subtask(X_train_sic8h, y_train_sic8h, X_valid_sic8h, valid_output_sic8h, SIC8h_event_valid, mode = 'mimic', type = 'binary', event_task='SIC 8h')

|MIMIC-Validation|====================================================


,Model,AUPRC,SIC score,Recall,Precision,Grecall
0,lgbm,0.480215,0.311111,0.333333,0.518519,0.264151
1,rf,0.714513,0.447504,0.888889,0.769231,0.377358
2,dt,0.586450,0.430075,0.444444,0.468085,0.415094
3,lr,0.366403,0.468625,0.444444,0.397059,0.509434


----------------------------------------------------------------------


In [144]:
result_sic8h.to_excel(excel_writer='sic8h_proposed_method.xlsx')